End to End Topic Modeling using NMF - Non-Negative Matrix Factorization

There are two ways of doing Topic Modeling

* LDA - it's statistical method
* NMF - it's linear algebraic method

Link to video that walks thorugh the code - https://youtu.be/25JOEnrz40c

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [5]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57448,2/29/2020,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I am attempting to make a payment toward my st...,None,"Nelnet, Inc.",KS,None,None,Consent provided,Web,2/29/2020,Closed with explanation,Yes,NaN,3549178
57449,2/11/2020,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,Received letter for {$480.00}. Original credit...,Company has responded to the consumer and the ...,"The Receivable Management Services LLC, New Yo...",AZ,853XX,None,Consent provided,Web,2/18/2020,Closed with explanation,Yes,NaN,3527928
57450,2/29/2020,Debt collection,Other debt,Communication tactics,"Used obscene, profane, or other abusive language",entire time 10 years until XX/XX/2020. XXXX ma...,Company has responded to the consumer and the ...,"Convergent Resources, Inc.",NJ,8101,None,Consent provided,Web,2/29/2020,Closed with explanation,Yes,NaN,3549238
57451,1/16/2020,Checking or savings account,Checking account,Problem with a lender or other company chargin...,Transaction was not authorized,I am a customer with Wells Fargo Bank. Recentl...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,AZ,852XX,None,Consent provided,Web,1/22/2020,Closed with explanation,Yes,NaN,3498566


In [6]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [7]:
df['Company'].value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
American Credit Systems, Inc                1
KMD Partners, LLC                           1
The Wilber Law Firm, P.C.                   1
BERKSHIRE HILLS BANCORP, INC                1
LPSG                                        1
Name: Company, Length: 2197, dtype: int64

In [8]:
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

In [9]:
pd.set_option('display.max_colwidth', -1)
complaints_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [10]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.3, random_state=111)

In [11]:
X_train['Product'].value_counts()

Debt collection                15281
Credit card or prepaid card    9263 
Mortgage                       6815 
Checking or savings account    4864 
Student loan                   2078 
Vehicle loan or lease          1916 
Name: Product, dtype: int64

In [12]:
#stemmer = PorterStemmer()
stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+','',word.strip('Xx/'))) > 3)] 
   #tokens = map(str.lower, tokens)
   str(tokens).lower()
   stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
   return tokens

In [14]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=0.75, max_features=1000, lowercase=False, ngram_range=(1,2))
tf_vectors = vectorizer_tf.fit_transform(X_train.complaints) 

In [15]:
tf_vectors.A

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.10041401, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
vectorizer_tf.get_feature_names()

['ACCOUNT',
 'Account',
 'After',
 'Again',
 'Also',
 'America',
 'American',
 'American Express',
 'Bank',
 'Bank America',
 'Because',
 'Bureau',
 'CFPB',
 'CREDIT',
 'Capital',
 'Card',
 'Chase',
 'Citi',
 'Citibank',
 'Collection',
 'Company',
 'Consumer',
 'Cooper',
 'Credit',
 'Credit Reporting',
 'Customer',
 'Date',
 'Debt',
 'Debt Collection',
 'Department',
 'Discover',
 'During',
 'Express',
 'FCRA',
 'FDCPA',
 'Fair',
 'Fair Credit',
 'Fair Debt',
 'Fargo',
 'Federal',
 'Financial',
 'First',
 'Fraud',
 'Freedom',
 'HAVE',
 'Home',
 'However',
 'Loan',
 'Monday',
 'Mortgage',
 'NEVER',
 'Navient',
 'Once',
 'Payment',
 'Please',
 'Please help',
 'Portfolio',
 'Practices',
 'Recovery',
 'Reporting',
 'Section',
 'Service',
 'Services',
 'Servicing',
 'Shellpoint',
 'Since',
 'State',
 'Synchrony',
 'THAT',
 'THEY',
 'THIS',
 'Thank',
 'That',
 'Their',
 'Then',
 'There',
 'These',
 'They',
 'They have',
 'They said',
 'This',
 'This company',
 'Today',
 'USAA',
 'WITH',
 'We

In [17]:
clf = decomposition.NMF(n_components=6,  random_state=111)

W1 = clf.fit_transform(tf_vectors)
H1 = clf.components_

In [18]:
H1

array([[0.        , 0.        , 0.23077248, ..., 0.17088861, 0.03647509,
        0.        ],
       [0.02938865, 0.113267  , 0.02249966, ..., 0.15892741, 0.4679712 ,
        0.16948754],
       [0.00925373, 0.00224646, 0.05525066, ..., 0.09322272, 0.05349518,
        0.        ],
       [0.        , 0.02946175, 0.1012087 , ..., 0.43141954, 0.25560736,
        0.00370241],
       [0.00411046, 0.01334859, 0.        , ..., 0.        , 0.02658514,
        0.05452322],
       [0.0333625 , 0.14681305, 0.07080417, ..., 0.03468337, 0.17411201,
        0.        ]])

In [19]:
W1

array([[0.03160029, 0.02422523, 0.        , 0.        , 0.        ,
        0.        ],
       [0.05852762, 0.        , 0.03658845, 0.01244973, 0.00094219,
        0.07209997],
       [0.        , 0.03639921, 0.00450408, 0.        , 0.00384169,
        0.06009649],
       ...,
       [0.03230101, 0.        , 0.02828272, 0.00106785, 0.        ,
        0.02868812],
       [0.        , 0.01761457, 0.02526676, 0.        , 0.00385552,
        0.00165738],
       [0.        , 0.        , 0.        , 0.07125372, 0.        ,
        0.        ]])

In [20]:
num_words=15

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [21]:
topics

['they that them told have called call would said with They back from phone that they',
 'debt credit this collection report credit report company have this debt from that with reporting information letter',
 'card credit credit card charges charge Capital cards balance Citi limit score that have report used',
 'payment loan payments mortgage late that have interest with made paid month been this loans',
 'theft identity theft identity victim belong attached does legal report this debt Please this debt information result',
 'account bank Bank checking closed check this that funds with checking account this account opened money America']

In [24]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [25]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.03,0.02,0.00,0.00,0.0,0.00,0
Doc1,0.06,0.00,0.04,0.01,0.0,0.07,5
Doc2,0.00,0.04,0.00,0.00,0.0,0.06,5
Doc3,0.04,0.00,0.00,0.00,0.0,0.01,0
Doc4,0.02,0.00,0.00,0.00,0.0,0.04,5
...,...,...,...,...,...,...,...
Doc40212,0.02,0.00,0.07,0.00,0.0,0.01,2
Doc40213,0.01,0.01,0.02,0.01,0.0,0.03,5
Doc40214,0.03,0.00,0.03,0.00,0.0,0.03,0
Doc40215,0.00,0.02,0.03,0.00,0.0,0.00,2


In [26]:
X_train.head()

complaints  ...                          Company
30936  Throughout XX/XX/2019, I have received multiple calls from this company. I have also asked them to not call me or anyone associated with me. They have called my mother in law way too many times asking for my partner. They have called my partner multiple times asking for me. I'm not sure what kind of operation is being ran there, but it needs to stop. I'm giving notice to cease all contact with me or anyone else except the creditor about this claimed debt. Only contact me in writing, not by phone. Thank you.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [27]:
WHold = clf.transform(vectorizer_tf.transform(X_hold.complaints[:5]))

In [28]:
colnames = ["Topic" + str(i) for i in range(clf.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complaints[:5]))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [29]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.03,0.00,0.00,0.00,0.0,0.09,5
Doc1,0.01,0.02,0.00,0.02,0.0,0.02,1
Doc2,0.02,0.00,0.03,0.00,0.0,0.06,5
Doc3,0.01,0.00,0.00,0.07,0.0,0.00,3
Doc4,0.01,0.00,0.03,0.03,0.0,0.02,2


In [30]:
X_hold.head()

,complaints,Product,Company
30060,"I have a business checking account at BB & T. On XX/XX/2019, I attempted to deposit a check into my account and I received a message stating that I was over my monthly mobile deposit limit. I was confused because it was the first of the month and I had not deposited any checks since the previous month. I called BB & T and they said that I couldnt deposit checks into business accounts via the mobile app even though I had done that before. \n\nI was instructed to open a personal account, into which I could deposit checks via the mobile app. I was told that if I opened the account online I would have immediate access, that I could link my personal and business accounts, and immediately be able to transfer money between them. \n\nOn XX/XX/XXXX, I opened my personal account online. Though I successfully opened online, I did not have online access as I had been promised. Because I was traveling in an area where there were no BB & T branches, I could not go into a branch until XX/XX/2019. In the intervening time, my business account became overdrawn. I had the money in my personal account to bring it back into the black, but BB & T could not make the transfer until I went into a branch. During this time, I incurred an astonishing {$320.00} in overdraft fees in my business account because I had no way of transferring money between the two accounts. \n\nWhen I went into the branch, I met with XXXX XXXX in XXXX XXXX County, Florida. She investigated the accounts and told me she would link them and that I would have online access in XXXX hours. The online access still never material, I was not able to transfer money and I received an additional {$36.00} overdraft fee. This brought the total to {$360.00} in overdraft fees. \n\nI had only opened the second account on BB & Ts advice ; I could have had my customer send a bank wire directly into business account in the first week of XXXX to keep the account positive. My TOTAL deposits between the two accounts was always positive. \n\nI requested a refund and was told they would not refund any of the fees.",Checking or savings account,BB&T CORPORATION
53473,"To who it may concern, My concern is regarding Shellpoint Mortgage Servicing Company. This company has received my monthly mortgage payments and have failed to account it as payments received. My bank has sent me proof the account was cleared and paid. This company is trying to foreclosed on my house and has sent me a letter indicating they sent my file to their Loss Mitigation dept. The person who they assigned it to is XXXX XXXX XXXX x XXXX. Their corporate number is XXXX. \n\nI have can send proof of my claim from above should you need it. Please assist.",Mortgage,"Shellpoint Partners, LLC"
35879,I contacted XXXX about fraudulent charges that were made on my account and the customer service representative told me that they wouldnt be able to issue anew card or remove my fraudulent charges since the account was closed due to non-payment. I was unaware of this these charging that were made.. To my knowledge I didnt owe a payment because the account wasnt being used.,Credit card or prepaid card,"EQUIFAX, INC."
20993,"I first applied for the Fedloan Serving program in XXXX in hopes of getting loan forgiveness since I've been paying on my student loans since late XXXX or early XXXX and I have been a public servant since XXXX, XXXX. I have never missed a payment to any of the various loan companies that owned my loans. The Fedloan Program turned me down in XXXX, saying my loans, which were originally Federal Stafford Loans, did not qualify me for the Program. In XXXX, XXXX, I was solicited by XXXX XXXX and told I could get into this program to reduce my loan payments. I was charged {$690.00} by XXXX XXXX to get me into the Fedloan Serving program. My loan payments went down to $ XXXX/month for the first year. In XXXX, XXXX, Fedloan Servicing informed me via email that I needed to recertify my "" income-drive

-----